In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [2]:
#0050
url = "https://www.yuantaetfs.com/api/StkWeights?date=&fundid=1066"
df = pd.read_json(url)
df.info()

#get 50 stock price
data = {}
for x in df['code']:
    stock_no = f"{x}.tw"
    data[stock_no] = yf.download(stock_no, start='2020-01-01')
    
#get 50 Adj Close  
df_adj = pd.DataFrame()
for x in data:
    df_adj[x] = data[x]['Adj Close']

#get stock index ^TWII
stockindex = yf.download('^TWII', start='2020-01-01')

#merge data
df = pd.DataFrame()
df = pd.merge(stockindex[['Adj Close']], df_adj, how = 'left', on = 'Date').dropna(axis=0)
df.rename(columns={'Adj Close':'TWII'},inplace=True)

#use each stock return runs a regression on index
ret_df = df.pct_change().dropna(axis=0)
final_output = {}
alpha_list = []
beta_list = []

for company in ret_df.columns[1:]:
    X = ret_df[['TWII']].assign(Intercept=1)
    Y = ret_df[company]
    X1 = sm.add_constant(X)     
    model = sm.OLS(Y, X1)
    results = model.fit()
    alpha = pd.read_html(results.summary().as_html(), header=0, index_col=0)[1].iloc[1][0]
    beta = pd.read_html(results.summary().as_html(), header=0, index_col=0)[1].iloc[0][0]
    final_output[company] = [alpha, beta]
    alpha_list.append(alpha)
    beta_list.append(beta)
    
#convert dict to DataFrame
final_output_df= pd.DataFrame.from_dict(final_output).T
final_output_df.columns = ['alpha',' beta']
final_output_df  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   code     50 non-null     int64  
 1   ym       0 non-null      float64
 2   name     50 non-null     object 
 3   ename    50 non-null     object 
 4   weights  50 non-null     float64
 5   qty      50 non-null     int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 2.5+ KB
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************

/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib

/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/jochien/opt/anaconda3/lib

,alpha,beta
2330.tw,0.000300,1.1705
2454.tw,0.000700,1.4366
2317.tw,-0.000087,0.9871
2303.tw,0.001600,1.2133
2308.tw,0.000500,1.0209
2891.tw,0.000200,0.7590
2881.tw,0.000700,0.8655
2882.tw,0.000400,0.7922
1301.tw,-0.000100,0.8254
2412.tw,0.000300,0.1648


In [3]:
ret_df.head()

,TWII,2330.tw,2454.tw,2317.tw,2303.tw,2308.tw,2891.tw,2881.tw,2882.tw,1301.tw,...,2395.tw,2409.tw,4904.tw,1590.tw,6505.tw,2801.tw,8046.tw,9910.tw,2408.tw,8454.tw
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,0.000822,0.001475,-0.015855,0.008811,-0.015106,0.003311,0.004474,0.006459,0.005875,0.019076,...,0.008375,-0.009852,0.000000,-0.031216,0.030612,0.002193,-0.026096,0.000000,-0.013953,0.000000
2020-01-06,-0.012970,-0.022091,-0.024166,-0.012009,-0.015337,-0.016502,-0.008909,-0.007487,-0.012850,0.004926,...,-0.008306,-0.008955,-0.004149,0.004444,0.009901,-0.013129,-0.045016,-0.002551,-0.012972,0.000000
2020-01-07,-0.006110,-0.007530,0.004717,-0.015470,-0.003115,0.000000,0.004494,-0.002155,-0.001183,-0.004902,...,0.006700,0.009036,0.000000,-0.015487,-0.019608,0.000000,0.019080,-0.002558,-0.031063,0.003448
2020-01-08,-0.005321,0.000000,0.010563,-0.029181,-0.015625,-0.006712,-0.004474,-0.010799,-0.009479,-0.014778,...,-0.004992,0.029851,-0.006944,-0.020225,-0.002000,-0.013304,-0.005507,-0.017949,0.018496,-0.001718
2020-01-09,0.012992,0.024279,0.015099,0.006936,0.019048,0.027027,0.002247,0.004367,0.004785,0.000000,...,0.020067,-0.019324,-0.006993,0.099771,-0.017034,0.006742,0.005537,0.007833,0.009685,-0.003442
